In [1]:
import scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import random

In [2]:
import time

In [3]:
def chol_psd(a):
    
    [row,col] = a.shape
    root = np.full([row,col], 0.0, dtype='float64')
    n = row
    
    for j in range(n):
        s = 0.0
        
        if j > 0:
            s = np.matmul(root[j,:j].T, root[j,:j])
            
        temp = a[j,j] - s
        if -1e-8 <= temp <= 0:
            temp = 0.0
        
        root[j,j] = np.sqrt(temp);
        
        if root[j,j] == 0:
            root[j,j+1:n] = 0.0
        else:
            ir = 1.0/root[j,j]
            for i in range(j+1,n):
                s = np.matmul(root[j,:j].T, root[i,:j])
                root[i,j] = (a[i,j] - s) * ir
    
    return np.matrix(root)

In [4]:
n = 500
sigma = np.matrix(np.full((n, n), 0.9), dtype='float64')
np.fill_diagonal(sigma, 1)
sigma[0, 1] = 1
sigma[1, 0] = 1

root = chol_psd(sigma)
(np.matmul(root, root.T) - sigma).sum()

-7.299705284680158e-11

In [5]:
def near_psd(a):
    epsilon = 0.0
    n = a.shape[0]
    
    invSD = None
    out = a.copy()
    
    if np.count_nonzero(np.diag(a) == 1) != n:
        invSD = np.diag(np.divide(1, np.sqrt(np.diag(a))))
        out = np.matmul(np.matmul(invSD, out), invSD)
    
    vals, vecs = np.linalg.eigh(out)
    vals = np.matrix(np.maximum(vals, epsilon))
    vecs = np.matrix(vecs)
    
    T = 1./np.matmul(np.multiply(vecs, vecs), vals.T)
    T = np.diag(np.sqrt(np.array(T).reshape(n)))
    l = np.diag(np.array(np.sqrt(vals)).reshape(n))
    B = np.matmul(np.matmul(T, vecs), l)
    out = np.matmul(B, B.T)
    
    if invSD != None:
        invSD = np.diag(np.divide(1, np.diag(invSD)))
        out = np.matmul(np.matnul(invSD, out), invSD)
    
    return out

In [6]:
n = 5
sigma = np.matrix(np.full((n, n), 0.9), dtype='float64')
np.fill_diagonal(sigma, 1)
sigma[0, 1] = 0.7357
sigma[1, 0] = 0.7357

root = near_psd(sigma)
(root -sigma).sum()

-3.965853847232026e-05

In [7]:
def getAplus(A):
    vals, vecs = np.linalg.eigh(A)
    vals = np.matrix(np.diag(np.maximum(vals, 0)))
    vecs = np.matrix(vecs)
    return vecs * vals * vecs.T

In [8]:
def getPs(A, W):
    W05 = np.sqrt(W)
    iW = W05.I
    return iW * getAplus(W05 * A * W05) * iW

In [9]:
def getPu(A, W):
    Aret = A.copy()
    for i in range(0, Aret.shape[0]):
        Aret[i, i] = 1.0
    return Aret

In [10]:
def wgtNorm(A, W):
    W05 = np.sqrt(W)
    W05 = W05 * A * W05
    return np.multiply(W05, W05).sum()

In [11]:
def higham_nearestPSD(pc, W = None, epsilion = 1e-9, maxIter = 100, tol = 1e-9):
    
    n = pc.shape[0]
    if W == None:
        W = np.matrix(np.diag(np.full(n, 1.0)))
    
    deltaS = 0
    
    Yk = pc.copy()
    norml = sys.maxsize
    i = 1
    
    while i <= maxIter:
        Rk = Yk - deltaS
        Xk = getPs(Rk, W)
        deltaS = Xk - Rk
        Yk= getPu(Xk, W)
        norm = wgtNorm(Yk - pc, W)
        
        temvals, temvecs = np.linalg.eigh(Yk)
        minEigVal = np.min(temvals)
        
        if ((norm - norml) < tol) and (minEigVal > -epsilion):
            break
        
        norml = norm
        i += 1
    
    if i < maxIter:
        print("Converged in "+ str(i) +" iterations.")
    else:
        println("Convergence failed after " + str(i-1) + " iterations")
    
    return Yk

In [12]:
n = 500
sigma = np.matrix(np.full((n, n), 0.9), dtype='float64')
np.fill_diagonal(sigma, 1)
sigma[0, 1] = 0.7357
sigma[1, 0] = 0.7357

W = np.matrix(np.diag(np.full(n,1.0)))

hpsd = higham_nearestPSD(sigma)
npsd = near_psd(sigma)
norm_hpsd = wgtNorm(hpsd - sigma, W)
norm_npsd = wgtNorm(npsd - sigma, W)
print("Distance near_psd() = " + str(norm_npsd))
print("Distance higham_nearestPSD() = " + str(norm_hpsd))

Converged in 26 iterations.
Distance near_psd() = 0.39378468350341517
Distance higham_nearestPSD() = 0.008036763056475597


In [13]:
n = 1000
sigma = np.matrix(np.full((n, n), 0.9), dtype='float64')
np.fill_diagonal(sigma, 1)
sigma[0, 1] = 0.7357
sigma[1, 0] = 0.7357

W = np.matrix(np.diag(np.full(n,1.0)))

hpsd = higham_nearestPSD(sigma)
npsd = near_psd(sigma)
norm_hpsd = wgtNorm(hpsd - sigma, W)
norm_npsd = wgtNorm(npsd - sigma, W)
print("Distance near_psd() = " + str(norm_npsd))
print("Distance higham_nearestPSD() = " + str(norm_hpsd))

Converged in 26 iterations.
Distance near_psd() = 0.7929738934432975
Distance higham_nearestPSD() = 0.008152133420977845


In [14]:
start = time.time()
hpsd = higham_nearestPSD(sigma)
end = time.time()
higam_times = end - start

start = time.time()
npsd = near_psd(sigma)
end = time.time()
near_times = end - start

print("n=500")
print("Higam Took: " + str(higam_times) + " seconds")
print("Near_PSD Took: " + str(near_times) + " seconds")

Converged in 26 iterations.
n=500
Higam Took: 11.551598072052002 seconds
Near_PSD Took: 0.15267682075500488 seconds


In [15]:
n = 1000
sigma = np.matrix(np.full((n, n), 0.9), dtype='float64')
np.fill_diagonal(sigma, 1)
sigma[0, 1] = 0.7357
sigma[1, 0] = 0.7357

start = time.time()
hpsd = higham_nearestPSD(sigma)
end = time.time()
higam_times = end - start

start = time.time()
npsd = near_psd(sigma)
end = time.time()
near_times = end - start

print("n=1000")
print("Higam Took: " + str(higam_times) + " seconds")
print("Near_PSD Took: " + str(near_times) + " seconds")

Converged in 26 iterations.
n=1000
Higam Took: 13.736309051513672 seconds
Near_PSD Took: 0.15064096450805664 seconds


# Problem 3

In [16]:
def simulateNormal(N, cov, mean = np.empty(shape = (0, 0)), seed = 1234):
    
    #Error Checking
    [n, m] = cov.shape
    if n != m:
        raise Exception("Covariance Matrix is not square (" + str(n) + "," + str(m) + ")")
    
    out = np.full([n, N], None, dtype='float64')
    
    #If the mean is missing then set to 0, otherwise use provided mean
    _mean = np.full(n, 0.0)
    m = mean.shape[0]
    if m != 0:
        if n != m:
            raise Exception("Mean " + str(m) + " is not the size of cov (" + str(n) + "," + str(n) + ")")
        _mean = mean.copy()
    
    #Take the root
    l = chol_psd(cov)
    
    #Generate needed random standard normals
    out = np.matrix(np.random.normal(0, 1, (n, N)))
    
    #apply the standard normals to the cholesky root
    out = (l * out)
    
    #Add the mean
    for i in range(n):
        out[:,i] = out[:,i] + _mean[i]
    
    return out

In [17]:
def simulate_pca(a, nsim, target, mean = np.empty(shape = (0, 0)), seed = 1234):
    n = a.shape[0]
    
    #If the mean is missing then set to 0, otherwise use provided mean
    _mean = np.full(n, 0.0)
    m = mean.shape[0]
    if m != 0:
        if n != m:
            raise Exception("Mean " + str(m) + " is not the size of cov (" + str(n) + "," + str(n) + ")")
        _mean = mean.copy()
    
    #Eigenvalue decomposition
    vals, vecs = np.linalg.eigh(a)
    vals = vals.real
    vecs = vecs.real
    #python returns values lowest to highest, flip them and the vectors
    vals = vals[::-1]
    vecs = vecs[::-1]
    
    tv = vals.sum()
    
    vals = np.maximum(vals, 0)
    
    cumm_val_explained = np.cumsum(vals) / tv
    i=0
    for i in range(len(vals)):
        if cumm_val_explained[i] < target:
            i += 1
        else:
            break
            
    vals = vals[0:i+1]
    vecs = vecs[:, :i+1]
    
    B = vecs @ np.diag(np.sqrt(vals))
    np.random.seed(1234)
    z = np.random.normal(size=(len(vals), nsim))
    return B @ z

In [18]:
def getCor(cov):
    cov_diag = np.diag(cov)
    invSD = np.diag(np.divide(1, np.sqrt(cov_diag)))
    cor = invSD * cov * invSD
    return cor

def getCov(var, cor):
    std = np.sqrt(var)
    n = len(var)
    cov = np.matrix(np.zeros((n,n)))
    for i in range(n):
        for j in range(n):
            cov[i,j] = cor[i,j] * std[i] * std[j]
            
    return cov

def ewm (x, exp_w, lamda):
    w = []
    sum_w = 0
    n = x.shape[1]
    for i in range(1, len(x.index)+1):
        w.append((1-lamda)*lamda**(i-1))
        sum_w = sum_w + w[i-1]
    for i in range(len(x.index)):   
        exp_w.append(w[i] / sum_w)
    
    
    cov_matrix = np.zeros([n,n])
    for i in range (len(x.index)):
        for j in range (n):
            x.iloc[i,j] = x.iloc[i,j] - np.mean(x.iloc[:,j])
    
            
    for i in range (n):
        for j in range (n):
            temp = exp_w * x.iloc[:,i]
            cov_matrix[i,j] = np.dot(temp, x.iloc[:, j])
            
    return cov_matrix

def Norms (cov, cov_psd):
    temp = cov - cov_psd
    temp = temp**2
    Sum = temp.sum()
    return Sum

In [19]:
df = pd.read_csv(r'DailyReturn.csv').iloc[:, 1:]

pear_cov = np.matrix(df.cov())
pear_var = df.var()
pear_cor = np.matrix(df.corr())

ewm_cov = np.matrix(ewm(df, [], 0.97))
ewm_var = np.diag(ewm_cov)
ewm_cor = getCor(ewm_cov)

pear_var_cor = getCov(pear_var, pear_cor)
pear_var_ewm_cor = getCov(pear_var, ewm_cor)
ewm_var_pear_cor = getCov(ewm_var, pear_cor)
ewm_var_cor = getCov(ewm_var, ewm_cor)

In [20]:
# Print Function
matrixType = ["EWMA", "EWMA_COR_PEARSON_STD", "PEARSON", "PEARSON_COR_EWMA_STD"]
simType = ["Full", "PCA=1", "PCA=0.75", "PCA=0.5"]

matrix = []
simulation = []
runtimes = []
norms = []

nsim = 25000
for sim in simType:
    for mat in matrixType:
        matrix.append(mat)
        simulation.append(sim)
        elapse = 0
        
        if mat == "PEARSON":
            c = pear_var_cor
        elif mat == "EWMA_COR_PEARSON_STD":
            c = pear_var_ewm_cor
        elif mat == "EWMA":
            c = ewm_var_cor 
        elif mat == "PEARSON_COR_EWMA_STD":
            c = ewm_var_pear_cor
            
        if sim == 'Full':
            start = time.time()
            s = simulateNormal(nsim, c)
            end = time.time()
            elapse = end - start
        elif sim == 'PCA=1':
            start = time.time()
            s = simulate_pca(c, nsim, 1)    
            end = time.time()
            elapse = end - start
        elif sim == 'PCA=0.75':
            start = time.time()
            s = simulate_pca(c, nsim, 0.75)    
            end = time.time()
            elapse = end - start
        elif sim == 'PCA=0.5':
            start = time.time()
            s = simulate_pca(c, nsim, 0.5)    
            end = time.time()
            elapse = end - start
        
        
        covar = np.cov(s)
        runtimes.append(elapse)
        norms.append(Norms(covar, c))

In [21]:
result = pd.DataFrame(list(zip(matrix, simulation, norms, runtimes,)), columns = ['Name','Simulation', 'Norm', 'RunTime'])
result

,Name,Simulation,Norm,RunTime
0,EWMA,Full,5.808264e-07,0.174197
1,EWMA_COR_PEARSON_STD,Full,7.721642e-07,0.116791
2,PEARSON,Full,1.003231e-06,0.093920
3,PEARSON_COR_EWMA_STD,Full,7.392774e-07,0.094156
4,EWMA,PCA=1,4.577813e-03,0.062893
5,EWMA_COR_PEARSON_STD,PCA=1,5.339868e-03,0.045191
6,PEARSON,PCA=1,6.758644e-03,0.075414
7,PEARSON_COR_EWMA_STD,PCA=1,5.921737e-03,0.078212
8,EWMA,PCA=0.75,4.588553e-03,0.013578
9,EWMA_COR_PEARSON_STD,PCA=0.75,5.356165e-03,0.011057
